# This Notebook is to check the quality of the data and perform cleaup process for next steps

In [ ]:
#import necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# read csv data
TDCJ_Data = pd.read_csv("High_Value_Dataset__August_2022.csv", dtype={'Offense Code': object},parse_dates=['Sentence Date','Offense Date'])
TDCJ_Data.head()

For this analysis we will use the Gender, Race, Age,County,Offense Code, TDCJ Offense,Sentence Date, Offense Date and Sentence (Years)

In [ ]:
TDCJ_Data = TDCJ_Data[["Gender", "Race", "Age","County","Offense Code", "TDCJ Offense","Sentence Date","Offense Date","Sentence (Years)"]]
TDCJ_Data.head()                       

In [ ]:
# Check the data type
TDCJ_Data.dtypes

In [ ]:
TDCJ_Data.info()

There are some null values in the Sentence Date. Let's remove those rows

In [ ]:
TDCJ_Data = TDCJ_Data.dropna()
TDCJ_Data.info()

### Check How Many Offence Code and Offence types are there

In [ ]:
TDCJ_Data["TDCJ Offense"].unique().size

In [ ]:
TDCJ_Data["Offense Code"].unique().size

In [ ]:
# Check How Many Offence Code and Offence types are there

print(TDCJ_Data[["TDCJ Offense","Offense Code"]].drop_duplicates().sort_values(by ="Offense Code").head(10))


As it is clear that the description is not consistent with the offence code. For simplycity lets consider the first two digits of the offence code and count how many unique offence type are there

In [ ]:
# make a copy of data for offence code and offence description
Offence_Code = TDCJ_Data[["TDCJ Offense","Offense Code"]].copy()
Offence_Code["ShortCode"] =  TDCJ_Data["Offense Code"].str.slice(0,2)
Offence_Code.head()

In [ ]:
Offence_Code = Offence_Code[["ShortCode","TDCJ Offense"]].\
groupby(["ShortCode","TDCJ Offense"]).\
size().\
reset_index(name="obs").\
sort_values(by =["ShortCode","obs"],ascending = [True, False]).\
groupby(["ShortCode"]).\
first().\
reset_index().drop("obs",axis =1)

Offence_Code

There are 37 unique offence type

In [ ]:
TDCJ_Data["ShortCode"] =  TDCJ_Data["Offense Code"].str.slice(0,2)
TDCJ_Data =  TDCJ_Data.drop(["TDCJ Offense","Offense Code"], axis =1)
TDCJ_Data =  pd.merge(TDCJ_Data,Offence_Code)
TDCJ_Data.head()


In [ ]:
(TDCJ_Data["Sentence Date"]- TDCJ_Data["Offense Date"])/np.timedelta64(1,"D").astype('int8')

In [ ]:
TDCJ_Data["Case Duration"] =  (TDCJ_Data["Sentence Date"]- TDCJ_Data["Offense Date"])/np.timedelta64(1,"D")
TDCJ_Data["Sentence Year"] = TDCJ_Data["Sentence Date"].dt.year
TDCJ_Data["Offense Year"] = TDCJ_Data["Offense Date"].dt.year
TDCJ_Data["Age_at_Offence"] =  TDCJ_Data["Age"]-(2022 - TDCJ_Data["Offense Year"])
TDCJ_Data.drop(["Sentence Date","Offense Date", "Age","ShortCode"], axis =1,inplace=True)
TDCJ_Data.head()

In [ ]:
TDCJ_Data.to_csv("TDCJ_Data_clean.csv", index = False)